In [0]:
#!!pip uninstall tensorflow
#!pip install tensorflow==2.1.0

In [0]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
PROJECT_DIR = "/content/drive/My Drive/MachineLearning/Capstone/"
DATA_FILE_NAME = 'Data/Input Data Synthetic.xlsx'
CLEANSED_FILE_DIR = 'Data/processed/'

In [0]:
#import basic libraies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string
import nltk
from sklearn.preprocessing import LabelEncoder
#nltk.download('stopwords')

import warnings
warnings.filterwarnings('ignore')

In [0]:
import keras
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, LSTM, Embedding, Dropout, Activation, Bidirectional, Flatten
from tensorflow.keras.models import Model, Sequential

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

In [0]:
def load_doc(filename):
  if(filename.endswith('.xlsx')):
    data_df = pd.read_excel(filename, lines=True)
  elif(filename.endswith('.csv')):
    data_df = pd.read_csv(filename, keep_default_na = False)
  return data_df

In [0]:
def encode_target(target):
  le = LabelEncoder()
  return le.fit_transform(target), le

def decode_prediction(pred, encoder):
  return encoder.inverse_transform(pred) 

In [0]:
cleansed_data_df = load_doc(PROJECT_DIR+CLEANSED_FILE_DIR+'cleansed_data.csv')
print('shape of Data : ', cleansed_data_df.shape)

shape of Data :  (5691, 5)


In [0]:
cleansed_data_df.describe()

,short_description,description,combined_desc,caller,assignment_group
count,5691,5691,5691,5691,5691
unique,4292,5691,5691,2356,52
top,,language browser explorer customer number tele...,access cannot access go need access support tr...,bpctwhsn kzqsbmtp,GRP_0
freq,297,1,1,208,2630


In [0]:
cleansed_data_df.head()

,short_description,description,combined_desc,caller,assignment_group
0,login issue,user employee manager checked user ad reset pa...,login issue user employee manager checked user...,spxjnwir pjlcoqds,GRP_0
1,outlook,received team not outlook calendar somebody ad...,outlook received team not outlook calendar som...,hmjdrvpb komuaywn,GRP_0
2,cant log,received cannot log,cant log received cannot log,eylqgodm ybqkwiam,GRP_0
3,unable access page,unable access page,unable access page unable access page,xbkucsvz gcpydteq,GRP_0
4,error,error,error error,owlgqjme qhcozdfx,GRP_0


In [0]:
cleansed_data_df['assignment_group'].value_counts()

GRP_0         2630
GRP_2          218
GRP_8          208
GRP_12         205
GRP_19         195
GRP_3          190
GRP_13         137
GRP_9          127
GRP_6          119
GRP_14         111
GRP_25         104
GRP_29          94
GRP_4           84
GRP_18          84
GRP_16          83
GRP_10          83
GRP_24          83
GRP_33          72
GRP_7           65
GRP_26          54
GRP_COMMON      52
GRP_34          51
GRP_40          45
GRP_5           41
GRP_28          41
GRP_41          39
GRP_15          38
GRP_20          36
GRP_31          32
GRP_22          31
GRP_11          30
GRP_45          28
GRP_21          28
GRP_42          27
GRP_23          24
GRP_62          22
GRP_47          20
GRP_1           19
GRP_37          16
GRP_27          15
GRP_50          13
GRP_44          13
GRP_39          12
GRP_36          11
GRP_65          10
GRP_53           9
GRP_52           9
GRP_55           8
GRP_51           7
GRP_46           6
GRP_30           6
GRP_59           6
Name: assign

In [0]:
cleansed_data_df['description_length'] = cleansed_data_df['description'].str.len()
desc_max_length = cleansed_data_df['description_length'].max()
maxlen_desc_idx = cleansed_data_df['description_length'].idxmax()
print('Desc with max length ({}) at index {}:\n{}'.format(desc_max_length,maxlen_desc_idx,cleansed_data_df['description'].loc[maxlen_desc_idx]))

Desc with max length (3139) at index 2788:
source system android user unknown location unknown status field user no event log incident overview seeing company device generating server response sinkhole set probable infected asset traffic not blocked port port android device host likely infected return traffic android likely visit domain give false information order prevent use domain address resolution sinkhole traffic possible indicator infected computer reaching controller taken law enforcement research organization part mitigation effort traffic sinkhole activity administrator may curious particular domain browse signature repeated sinkhole however clear indication infection incident via high priority ticket per default would like us handle differently future see handling let us know either corresponding us via ticket ticket back soc calling us ticket sinkhole domain explicit notification via medium priority ticket no phone call auto resolve sinkhole domain directly portal no explic

**Train & Test Data:**

In [0]:
X = cleansed_data_df['description']
y = cleansed_data_df.assignment_group
y, le = encode_target(y)

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.1, stratify=y, random_state = 10)

In [0]:
X_train.head()

4878               catalogue not working unable order see
2434    one workman whose user id not able login ess u...
1028      error message trying log order user id attached
3226                                  outlook not loading
5499    language browser explorer customer number tele...
Name: description, dtype: object

In [0]:
print('Shape of X_train : ',X_train.shape)
print('Shape of X_test : ',X_test.shape)
print('Shape of y_train : ',y_train.shape)
print('Shape of y_test : ',y_test.shape)

Shape of X_train :  (5121,)
Shape of X_test :  (570,)
Shape of y_train :  (5121,)
Shape of y_test :  (570,)


In [0]:
#pd.DataFrame(data = y_test.assignment_group.value_counts())

In [0]:
max_features = 10000
maxlen = cleansed_data_df['description_length'].max()
embedding_size = 200

**TF-IDF Vectors as features :**

In [0]:
unigram_tokenizer = TfidfVectorizer(ngram_range=(1,1), max_features = max_features).fit(X)
X_train_unigram_seq = unigram_tokenizer.transform(X_train)
X_test_unigram_seq = unigram_tokenizer.transform(X_test)
print('Vocabulary size of unigram :', len(unigram_tokenizer.get_feature_names()))

bigram_tokenizer = TfidfVectorizer(ngram_range=(1,2), max_features = max_features).fit(X)
X_train_bigram_seq = bigram_tokenizer.transform(X_train)
X_test_bigram_seq = bigram_tokenizer.transform(X_test)
print('Vocabulary size of bigram :', len(bigram_tokenizer.get_feature_names()))

trigram_tokenizer = TfidfVectorizer(ngram_range=(1,3), max_features = max_features).fit(X)
X_train_trigram_seq = trigram_tokenizer.transform(X_train)
X_test_trigram_seq = trigram_tokenizer.transform(X_test)
print('Vocabulary size of trigram :', len(trigram_tokenizer.get_feature_names()))

Vocabulary size of unigram : 3357
Vocabulary size of bigram : 10000
Vocabulary size of trigram : 10000


In [0]:
print('Shape of X_train_unigram_seq : ',X_train_unigram_seq.shape)
print('Shape of X_test_unigram_seq : ',X_test_unigram_seq.shape)
print('Shape of X_train_bigram_seq : ',X_train_bigram_seq.shape)
print('Shape of X_test_bigram_seq : ',X_test_bigram_seq.shape)
print('Shape of X_train_trigram_seq : ',X_train_trigram_seq.shape)
print('Shape of X_test_trigram_seq : ',X_test_trigram_seq.shape)

Shape of X_train_unigram_seq :  (5121, 3357)
Shape of X_test_unigram_seq :  (570, 3357)
Shape of X_train_bigram_seq :  (5121, 10000)
Shape of X_test_bigram_seq :  (570, 10000)
Shape of X_train_trigram_seq :  (5121, 10000)
Shape of X_test_trigram_seq :  (570, 10000)


In [0]:
#X_train_unigram_seq_pad = pad_sequences(X_train_seq, maxlen = 4164)
#X_test_unigram_seq_pad = pad_sequences(X_train_seq, maxlen = 4164)

In [0]:
#above sequences can be used for modeling